In [7]:
""" This module draw rectangle around the nearest face that would be detected and follow it """

import cv2
import dlib
import numpy as np

class FaceTracker:
    def __init__(self):
        self.video = cv2.VideoCapture(0)
        self.video.set(3, 1080)  # Set width
        self.video.set(4, 1920)  # Set height
        self.face_detector = dlib.get_frontal_face_detector()  # dlib's function
        self.current_tracked_face = None  # Face hasn't been traced yet

    def find_nearest_face(self, faces):
        """
        This method calculates squares of each face detected by webcam
        :param faces: Each face that would be detected
        :return: Value that has the biggest square among other in list
        """
        squares = [abs(face.right() - face.left()) * abs(face.bottom() - face.top()) for face in faces]
        nearest_idx = np.argmax(squares)
        return faces[nearest_idx]

    def draw_rectangle(self, face, image):
        """
        This function draws rectangle around the nearest face that has the biggest square around it
        :param face: List with coordinates around face
        :param image: Object that store an imageq
        :return: None
        """
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(img=image, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
        cv2.imshow(winname='Face Tracking', mat=image)

    def track_nearest_face(self):
        try:
            while True:
                ret, frame = self.video.read()
                if not ret:
                    break

                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = self.face_detector(gray)
                # while True:
                if len(faces) > 0:
                    nearest_face = self.find_nearest_face(faces)
                    if self.current_tracked_face is None:  # The face hasn't been tracked yet
                        self.current_tracked_face = nearest_face
                    else:
                        tracked_x, tracked_y, _, _ = self.current_tracked_face.left(), self.current_tracked_face.top(), self.current_tracked_face.width(), self.current_tracked_face.height()
                        new_x, new_y, _, _ = nearest_face.left(), nearest_face.top(), nearest_face.width(), nearest_face.height()
                        distance = np.sqrt((tracked_x - new_x)**2 + (tracked_y - new_y)**2)
                        if distance < 100:  # Adjust this threshold as needed
                            self.current_tracked_face = nearest_face

                    self.draw_rectangle(self.current_tracked_face, frame)
                elif len(faces) == 0:
                    cv2.imshow('Face Tracking', frame)
                    
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        except Exception as e:
            print(f'Exception: {e}')
        finally:
            self.video.release()
            cv2.destroyAllWindows()

            
if __name__ == '__main__':
    tracker = FaceTracker()
    tracker.track_nearest_face()